In [1]:
using JuMP
import Gurobi
import HiGHS
import Printf

# Benders decomposition

https://jump.dev/JuMP.jl/stable/tutorials/algorithms/benders_decomposition/#In-place-iterative-method 

## Monolithic problem

In [41]:
G = [
    0 3 2 2 0 0 0 0
    0 0 0 0 5 1 0 0
    0 0 0 0 1 3 1 0
    0 0 0 0 0 1 0 0
    0 0 0 0 0 0 0 4
    0 0 0 0 0 0 0 2
    0 0 0 0 0 0 0 4
    0 0 0 0 0 0 0 0
]
n = size(G, 1)
model = Model(HiGHS.Optimizer)
set_silent(model)
@variable(model, x[1:n, 1:n], Bin)
@variable(model, y[1:n, 1:n] >= 0)
@constraint(model, sum(x) <= 11)
@constraint(model, [i = 1:n, j = 1:n], y[i, j] <= G[i, j] * x[i, j])
@constraint(model, [i = 2:n-1], sum(y[i, :]) == sum(y[:, i]))
@objective(model, Min, 0.1 * sum(x) - sum(y[1, :]))
optimize!(model)
solution_summary(model)

solution_summary(; result = 1, verbose = false)
├ solver_name          : HiGHS
├ Termination
│ ├ termination_status : OPTIMAL
│ ├ result_count       : 1
│ ├ raw_status         : kHighsModelStatusOptimal
│ └ objective_bound    : -5.10000e+00
├ Solution (result = 1)
│ ├ primal_status        : FEASIBLE_POINT
│ ├ dual_status          : NO_SOLUTION
│ ├ objective_value      : -5.10000e+00
│ ├ dual_objective_value : NaN
│ └ relative_gap         : 0.00000e+00
└ Work counters
  ├ solve_time (sec)   : 6.53315e-03
  ├ simplex_iterations : 15
  ├ barrier_iterations : -1
  └ node_count         : 1

In [42]:
objective_value(model)

-5.1

In [43]:
function optimal_flows(x)
    return [(i, j) => x[i, j] for i in 1:n for j in 1:n if x[i, j] > 0]
end

monolithic_solution = optimal_flows(value.(y))

9-element Vector{Pair{Tuple{Int64, Int64}, Float64}}:
 (1, 2) => 3.0
 (1, 3) => 2.0
 (1, 4) => 1.0
 (2, 5) => 3.0
 (3, 5) => 1.0
 (3, 6) => 1.0
 (4, 6) => 1.0
 (5, 8) => 4.0
 (6, 8) => 2.0

## Iterative method

In [53]:
M = -sum(G)
model = Model(HiGHS.Optimizer)
set_silent(model)
@variable(model, x[1:n, 1:n], Bin)
@variable(model, θ >= M)
@constraint(model, sum(x) <= 11)
@objective(model, Min, 0.1 * sum(x) + θ)
model

A JuMP Model
├ solver: HiGHS
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 65
├ num_constraints: 66
│ ├ AffExpr in MOI.LessThan{Float64}: 1
│ ├ VariableRef in MOI.GreaterThan{Float64}: 1
│ └ VariableRef in MOI.ZeroOne: 64
└ Names registered in the model
  └ :x, :θ

In [54]:
function solve_subproblem(x_bar)
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[i in 1:n, j in 1:n] == x_bar[i, j])
    @variable(model, y[1:n, 1:n] >= 0)
    @constraint(model, [i = 1:n, j = 1:n], y[i, j] <= G[i, j] * x[i, j])
    @constraint(model, [i = 2:n-1], sum(y[i, :]) == sum(y[:, i]))
    @objective(model, Min, -sum(y[1, :]))
    optimize!(model)
    assert_is_solved_and_feasible(model; dual = true)
    return (obj = objective_value(model), y = value.(y), π = reduced_cost.(x))
end

solve_subproblem (generic function with 2 methods)

In [55]:
function print_iteration(k, args...)
    f(x) = Printf.@sprintf("%12.4e", x)
    println(lpad(k, 9), " ", join(f.(args), " "))
    return
end

print_iteration (generic function with 1 method)

In [56]:
MAXIMUM_ITERATIONS = 100

100

In [57]:
ABSOLUTE_OPTIMALITY_GAP = 1e-6

1.0e-6

In [58]:
println("Iteration  Lower Bound  Upper Bound          Gap")
for k in 1:MAXIMUM_ITERATIONS
    optimize!(model)
    assert_is_solved_and_feasible(model)
    lower_bound = objective_value(model)
    x_k = value.(x)
    ret = solve_subproblem(x_k)
    upper_bound = (objective_value(model) - value(θ)) + ret.obj
    gap = abs(upper_bound - lower_bound) / abs(upper_bound)
    print_iteration(k, lower_bound, upper_bound, gap)
    if gap < ABSOLUTE_OPTIMALITY_GAP
        println("Terminating with the optimal solution")
        break
    end
    cut = @constraint(model, θ >= ret.obj + sum(ret.π .* (x .- x_k)))
    @info "Adding the cut $(cut)"
end

Iteration  Lower Bound  Upper Bound          Gap
        1  -2.9000e+01   0.0000e+00          Inf
        2  -6.7000e+00   3.0000e-01   2.3333e+01
        3  -6.5000e+00   5.0000e-01   1.4000e+01
        4  -6.2000e+00  -4.2000e+00   4.7619e-01
        5  -6.1000e+00  -4.1000e+00   4.8780e-01
        6  -6.1000e+00  -4.1000e+00   4.8780e-01
        7  -5.1000e+00  -5.1000e+00   0.0000e+00
Terminating with the optimal solution


[ Info: Adding the cut 3 x[1,2] + 2 x[1,3] + 2 x[1,4] + θ ≥ 0
[ Info: Adding the cut 5 x[2,5] + x[3,5] + x[2,6] + 3 x[3,6] + x[4,6] + x[3,7] + θ ≥ 0
[ Info: Adding the cut x[3,7] + 4 x[5,8] + 2 x[6,8] + θ ≥ 0
[ Info: Adding the cut 3 x[1,2] + x[3,5] + 2 x[6,8] + 4 x[7,8] + θ ≥ 0
[ Info: Adding the cut 3 x[1,2] + x[3,5] + 3 x[3,6] + x[4,6] + x[3,7] + θ ≥ 0
[ Info: Adding the cut 3 x[1,2] + 2 x[1,3] + x[4,6] + θ ≥ 0


In [59]:
optimize!(model)
assert_is_solved_and_feasible(model)
x_optimal = value.(x)
optimal_ret = solve_subproblem(x_optimal)
iterative_solution = optimal_flows(optimal_ret.y)

9-element Vector{Pair{Tuple{Int64, Int64}, Float64}}:
 (1, 2) => 3.0
 (1, 3) => 2.0
 (1, 4) => 1.0
 (2, 5) => 3.0
 (3, 5) => 1.0
 (3, 6) => 1.0
 (4, 6) => 1.0
 (5, 8) => 4.0
 (6, 8) => 2.0

In [60]:
iterative_solution == monolithic_solution

true

In [61]:
objective_value(model)

-5.1

## Callback

In [62]:
optimizer = Gurobi.Optimizer
lazy_model = Model(optimizer)
set_silent(lazy_model)
@variable(lazy_model, x[1:n, 1:n], Bin)
@variable(lazy_model, θ >= M)
@constraint(lazy_model, sum(x) <= 11)
@objective(lazy_model, Min, 0.1 * sum(x) + θ)
lazy_model

Set parameter Username
Academic license - for non-commercial use only - expires 2026-02-19


A JuMP Model
├ solver: Gurobi
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 65
├ num_constraints: 66
│ ├ AffExpr in MOI.LessThan{Float64}: 1
│ ├ VariableRef in MOI.GreaterThan{Float64}: 1
│ └ VariableRef in MOI.ZeroOne: 64
└ Names registered in the model
  └ :x, :θ

In [63]:
number_of_subproblem_solves = 0
function my_callback(cb_data)
    status = callback_node_status(cb_data, lazy_model)
    if status != MOI.CALLBACK_NODE_STATUS_INTEGER
        # Only add the constraint if `x` is an integer feasible solution
        return
    end
    x_k = callback_value.(cb_data, x)
    θ_k = callback_value(cb_data, θ)
    global number_of_subproblem_solves += 1
    ret = solve_subproblem(x_k)
    if θ_k < (ret.obj - 1e-6)
        # Only add the constraint if θ_k violates the constraint
        cut = @build_constraint(θ >= ret.obj + sum(ret.π .* (x .- x_k)))
        MOI.submit(lazy_model, MOI.LazyConstraint(cb_data), cut)
    end
    return
end

set_attribute(lazy_model, MOI.LazyConstraintCallback(), my_callback)

In [64]:
optimize!(lazy_model)
assert_is_solved_and_feasible(lazy_model)

In [65]:
number_of_subproblem_solves

17

In [66]:
x_optimal = value.(x)
optimal_ret = solve_subproblem(x_optimal)
callback_solution = optimal_flows(optimal_ret.y)

9-element Vector{Pair{Tuple{Int64, Int64}, Float64}}:
 (1, 2) => 3.0
 (1, 3) => 2.0
 (1, 4) => 1.0
 (2, 5) => 3.0
 (3, 5) => 1.0
 (3, 6) => 1.0
 (4, 6) => 1.0
 (5, 8) => 4.0
 (6, 8) => 2.0

## In-place iterative method

In [67]:
model = Model(HiGHS.Optimizer)
set_silent(model)
@variable(model, x[1:n, 1:n], Bin)
@variable(model, θ >= M)
@constraint(model, sum(x) <= 11)
@objective(model, Min, 0.1 * sum(x) + θ)
model

A JuMP Model
├ solver: HiGHS
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 65
├ num_constraints: 66
│ ├ AffExpr in MOI.LessThan{Float64}: 1
│ ├ VariableRef in MOI.GreaterThan{Float64}: 1
│ └ VariableRef in MOI.ZeroOne: 64
└ Names registered in the model
  └ :x, :θ

In [68]:
subproblem = Model(HiGHS.Optimizer)
set_silent(subproblem)
@variable(subproblem, x_copy[i in 1:n, j in 1:n])
@variable(subproblem, y[1:n, 1:n] >= 0)
@constraint(subproblem, [i = 1:n, j = 1:n], y[i, j] <= G[i, j] * x_copy[i, j])
@constraint(subproblem, [i = 2:n-1], sum(y[i, :]) == sum(y[:, i]))
@objective(subproblem, Min, -sum(y[1, :]))
subproblem

A JuMP Model
├ solver: HiGHS
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 128
├ num_constraints: 134
│ ├ AffExpr in MOI.EqualTo{Float64}: 6
│ ├ AffExpr in MOI.LessThan{Float64}: 64
│ └ VariableRef in MOI.GreaterThan{Float64}: 64
└ Names registered in the model
  └ :x_copy, :y

In [69]:
function solve_subproblem(model, x)
    fix.(model[:x_copy], x)
    optimize!(model)
    assert_is_solved_and_feasible(model; dual = true)
    return (
        obj = objective_value(model),
        y = value.(model[:y]),
        π = reduced_cost.(model[:x_copy]),
    )
end

solve_subproblem (generic function with 2 methods)

In [70]:
println("Iteration  Lower Bound  Upper Bound          Gap")
for k in 1:MAXIMUM_ITERATIONS
    optimize!(model)
    assert_is_solved_and_feasible(model)
    lower_bound = objective_value(model)
    x_k = value.(x)
    ret = solve_subproblem(subproblem, x_k)
    upper_bound = (objective_value(model) - value(θ)) + ret.obj
    gap = abs(upper_bound - lower_bound) / abs(upper_bound)
    print_iteration(k, lower_bound, upper_bound, gap)
    if gap < ABSOLUTE_OPTIMALITY_GAP
        println("Terminating with the optimal solution")
        break
    end
    cut = @constraint(model, θ >= ret.obj + sum(ret.π .* (x .- x_k)))
    @info "Adding the cut $(cut)"
end

Iteration  Lower Bound  Upper Bound          Gap
        1  -2.9000e+01   0.0000e+00          Inf
        2  -6.7000e+00   3.0000e-01   2.3333e+01
        3  -6.5000e+00   5.0000e-01   1.4000e+01
        4  -6.2000e+00  -4.2000e+00   4.7619e-01
        5  -6.1000e+00  -4.1000e+00   4.8780e-01
        6  -6.1000e+00  -4.1000e+00   4.8780e-01
        7  -5.1000e+00  -5.1000e+00   0.0000e+00
Terminating with the optimal solution


[ Info: Adding the cut 3 x[1,2] + 2 x[1,3] + 2 x[1,4] + θ ≥ 0
[ Info: Adding the cut 5 x[2,5] + x[3,5] + x[2,6] + 3 x[3,6] + x[4,6] + x[3,7] + θ ≥ 0
[ Info: Adding the cut x[3,7] + 4 x[5,8] + 2 x[6,8] + θ ≥ 0
[ Info: Adding the cut 3 x[1,2] + x[3,5] + 2 x[6,8] + 4 x[7,8] + θ ≥ 0
[ Info: Adding the cut 3 x[1,2] + x[3,5] + 3 x[3,6] + x[4,6] + x[3,7] + θ ≥ 0
[ Info: Adding the cut 3 x[1,2] + 2 x[1,3] + x[4,6] + θ ≥ 0


In [71]:
optimize!(model)
assert_is_solved_and_feasible(model)
x_optimal = value.(x)
optimal_ret = solve_subproblem(subproblem, x_optimal)
inplace_solution = optimal_flows(optimal_ret.y)

9-element Vector{Pair{Tuple{Int64, Int64}, Float64}}:
 (1, 2) => 3.0
 (1, 3) => 2.0
 (1, 4) => 1.0
 (2, 5) => 3.0
 (3, 5) => 1.0
 (3, 6) => 1.0
 (4, 6) => 1.0
 (5, 8) => 4.0
 (6, 8) => 2.0

In [72]:
inplace_solution == monolithic_solution

true

## Feasibility cuts

In [73]:
model = Model(HiGHS.Optimizer)
set_silent(model)
@variable(model, x[1:n, 1:n], Bin)
@variable(model, θ >= M)
@constraint(model, sum(x) <= 11)
@objective(model, Min, 0.1 * sum(x) + θ)
model

A JuMP Model
├ solver: HiGHS
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 65
├ num_constraints: 66
│ ├ AffExpr in MOI.LessThan{Float64}: 1
│ ├ VariableRef in MOI.GreaterThan{Float64}: 1
│ └ VariableRef in MOI.ZeroOne: 64
└ Names registered in the model
  └ :x, :θ

In [74]:
subproblem = Model(HiGHS.Optimizer)
set_silent(subproblem)
# We need to turn presolve off so that HiGHS will return an infeasibility
# certificate.
set_attribute(subproblem, "presolve", "off")
@variable(subproblem, x_copy[i in 1:n, j in 1:n])
@variable(subproblem, y[1:n, 1:n] >= 0)
@constraint(subproblem, sum(y) >= 1)  # <--- THIS IS NEW
@constraint(subproblem, [i = 1:n, j = 1:n], y[i, j] <= G[i, j] * x_copy[i, j])
@constraint(subproblem, [i = 2:n-1], sum(y[i, :]) == sum(y[:, i]))
@objective(subproblem, Min, -sum(y[1, :]))
subproblem

A JuMP Model
├ solver: HiGHS
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 128
├ num_constraints: 135
│ ├ AffExpr in MOI.EqualTo{Float64}: 6
│ ├ AffExpr in MOI.GreaterThan{Float64}: 1
│ ├ AffExpr in MOI.LessThan{Float64}: 64
│ └ VariableRef in MOI.GreaterThan{Float64}: 64
└ Names registered in the model
  └ :x_copy, :y

In [75]:
function solve_subproblem_with_feasibility(model, x)
    fix.(model[:x_copy], x)
    optimize!(model)
    if is_solved_and_feasible(model; dual = true)
        return (
            is_feasible = true,
            obj = objective_value(model),
            y = value.(model[:y]),
            π = reduced_cost.(model[:x_copy]),
        )
    end
    return (
        is_feasible = false,
        v = dual_objective_value(model),
        u = reduced_cost.(model[:x_copy]),
    )
end

solve_subproblem_with_feasibility (generic function with 1 method)

In [76]:
println("Iteration  Lower Bound  Upper Bound          Gap")
for k in 1:MAXIMUM_ITERATIONS
    optimize!(model)
    assert_is_solved_and_feasible(model)
    lower_bound = objective_value(model)
    x_k = value.(x)
    ret = solve_subproblem_with_feasibility(subproblem, x_k)
    if ret.is_feasible
        # Benders Optimality Cuts
        upper_bound = (objective_value(model) - value(θ)) + ret.obj
        gap = abs(upper_bound - lower_bound) / abs(upper_bound)
        print_iteration(k, lower_bound, upper_bound, gap)
        if gap < ABSOLUTE_OPTIMALITY_GAP
            println("Terminating with the optimal solution")
            break
        end
        @constraint(model, θ >= ret.obj + sum(ret.π .* (x .- x_k)))
    else
        # Benders Feasibility Cuts
        cut = @constraint(model, ret.v + sum(ret.u .* (x .- x_k)) <= 0)
        @info "Adding the feasibility cut $(cut)"
    end
end

Iteration  Lower Bound  Upper Bound          Gap


[ Info: Adding the feasibility cut -3 x[1,2] - 2 x[1,3] - 2 x[1,4] - 5.000000000000002 x[2,5] - x[3,5] - 2 x[2,6] - 6 x[3,6] - 2 x[4,6] - 2.0000000000000004 x[3,7] - 4 x[5,8] ≤ -1
[ Info: Adding the feasibility cut -2.999999999999999 x[1,2] - 1.9999999999999996 x[1,3] - 1.9999999999999996 x[1,4] - 10 x[2,5] - 2 x[3,5] - 1.9999999999999996 x[2,6] - 5.999999999999998 x[3,6] - 1.9999999999999996 x[4,6] - 1.9999999999999998 x[3,7] ≤ -0.9999999999999998
[ Info: Adding the feasibility cut -2.9999999999999987 x[1,2] - 5.999999999999998 x[1,3] - 1.9999999999999991 x[1,4] - 10 x[2,5] - 1.9999999999999991 x[2,6] - 1.9999999999999991 x[4,6] ≤ -0.9999999999999996
[ Info: Adding the feasibility cut -4 x[1,3] - 3.9999999999999982 x[1,4] - 9.999999999999998 x[2,5] - 2 x[2,6] - 4 x[5,8] - 1.9999999999999996 x[6,8] - 4 x[7,8] ≤ -1
[ Info: Adding the feasibility cut -3 x[1,2] - 4 x[1,3] - 2 x[1,4] - 10 x[2,5] - x[3,5] - 4 x[6,8] - 4 x[7,8] ≤ -1
[ Info: Adding the feasibility cut -6 x[1,2] - 6 x[1,3] - 4

       29  -2.8700e+01  -7.0000e-01   4.0000e+01
       33  -1.1200e+01  -2.0000e-01   5.5000e+01


[ Info: Adding the feasibility cut -3 x[1,2] - 2 x[1,4] - 3 x[3,6] - 3 x[3,7] - 12 x[5,8] - 4 x[6,8] ≤ -1
[ Info: Adding the feasibility cut -3.000000000000023 x[1,2] - 4.000000000000014 x[1,3] - 4.9999999999999964 x[2,5] - 2.000000000000007 x[2,6] - 2.9999999999999956 x[3,6] - 3.000000000000006 x[4,6] - 4.000000000000077 x[5,8] - 2.3092638912203256e-14 x[6,8] - 3.9999999999999933 x[7,8] ≤ -1.0000000000000078
[ Info: Adding the feasibility cut -2 x[1,3] - 10 x[2,5] - x[3,5] - 3 x[2,6] - 6 x[3,6] - 3 x[4,6] - 2 x[3,7] - 4 x[5,8] ≤ -1
[ Info: Adding the feasibility cut -10 x[2,5] - 2 x[3,5] - 3 x[2,6] - 9 x[3,6] - 3 x[4,6] - 3 x[3,7] - 4 x[5,8] ≤ -1
[ Info: Adding the feasibility cut -6 x[1,4] - 10 x[2,5] - 2 x[3,5] - 3 x[2,6] - 9 x[3,6] - 3 x[3,7] - 4 x[5,8] ≤ -1
[ Info: Adding the feasibility cut -6 x[1,4] - 10 x[2,5] - 2 x[3,5] - 3 x[2,6] - 9 x[3,6] - 4 x[5,8] - 12 x[7,8] ≤ -1
[ Info: Adding the feasibility cut -6 x[1,4] - 3 x[2,6] - 9 x[3,6] - 12 x[5,8] - 12 x[7,8] ≤ -1
[ Info: Addin

       34  -6.4000e+00  -2.4000e+00   1.6667e+00
       35  -6.2000e+00  -3.2000e+00   9.3750e-01
       36  -6.1000e+00  -4.1000e+00   4.8780e-01
       37  -5.2000e+00  -4.2000e+00   2.3810e-01
       38  -5.1000e+00  -2.1000e+00   1.4286e+00
       39  -5.1000e+00  -4.1000e+00   2.4390e-01
       40  -5.1000e+00  -5.1000e+00   0.0000e+00
Terminating with the optimal solution


In [77]:
optimize!(model)
assert_is_solved_and_feasible(model)
x_optimal = value.(x)
optimal_ret = solve_subproblem(subproblem, x_optimal)
feasible_inplace_solution = optimal_flows(optimal_ret.y)

9-element Vector{Pair{Tuple{Int64, Int64}, Float64}}:
 (1, 2) => 3.0
 (1, 3) => 2.0
 (1, 4) => 1.0
 (2, 5) => 3.0
 (3, 5) => 1.0
 (3, 6) => 1.0
 (4, 6) => 1.0
 (5, 8) => 4.0
 (6, 8) => 2.0

In [78]:
feasible_inplace_solution == monolithic_solution

true